In [99]:
# 出走馬について上位に入る確率を予想し、確率が閾値以上の場合に限り購入することを考える
# 閾値を使わずに購入した場合と閾値以上の場合で比較し、的中率や回収率が改善されるか確認する

In [3]:
# インポート
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
# ! pip install japanize-matplotlib
import japanize_matplotlib
from scipy import stats


def confidence_interval(mean: float, std: float, n: int, confidence: float = 0.95):

    """
    回収率の信頼区間を求める関数

    Parameters
    ----------
    mean       : float           平均回収率（例：1.05）
    std        : float           標準偏差  （例：0.15）
    n          : int             サンプル数（レース数）
    confidence : float, optional 信頼水準（デフォルトは0.95）

    Returns
    -------
    (lower, upper) : tuple of float
        信頼区間の下限値と上限値
    """
    
    # t値（t分布の臨界値）
    t_value = stats.t.ppf((1 + confidence) / 2, df=n-1)
    
    # 信頼区間の幅（マージン）
    margin = t_value * std / np.sqrt(n)
    
    # 区間の下限・上限
    lower = mean - margin
    upper = mean + margin
    
    return lower, upper

def handle_missing_val(df, features, enabled, min_horses):
    if not enabled:
        print("⚠ 欠損処理はスキップされました。")
        return df

    print("✅ 欠損処理を開始します...")
    valid_race_ids = []
    processed_groups = []

    for race_id, group in df.groupby('race_id'):
        # 欠損がない行数をカウント
        non_missing_mask = group[features].notnull().all(axis=1)
        non_missing_count = non_missing_mask.sum()

        if non_missing_count < min_horses:
            # → 指定頭数未満 → 除外
            continue
        else:
            # → 指定頭数以上 → 平均値で補完
            for col in features:
                if group[col].isnull().any():
                    mean_val = group[col].mean(skipna=True)
                    group[col] = group[col].fillna(mean_val)
            processed_groups.append(group)

    df_processed = pd.concat(processed_groups, ignore_index=True)
    print(f"✅ 欠損処理後のレコード数: {len(df_processed)}（元: {len(df)}）")
    return df_processed

# CSVファイルの読み込み
num_horses = 12
df = pd.read_csv(f'input_{num_horses}頭_編集後.csv', encoding='utf-8')
df['race_date'] = pd.to_datetime(df['race_date'])  # 日付が文字列なら変換
print(f"dfの行数は {len(df)} ")
# レースID、馬番順に並び替え
df = df.sort_values(by=['race_id', 'horse_number'])
# === 欠損値対応の設定 ===
handle_missing = True  # Trueで対応ON、Falseでスキップ
min_non_missing_horses = num_horses  # この頭数以上であれば補完、それ未満ならレースごと除外

# 欠損処理対象の特徴量を指定（必要に応じて拡張）
missing_check_features = [
    'time_index_average_2',
    'jockey_place_rate_100'
]

# === 欠損処理の実行 ===
df_not_handle_missing_val = df.copy()
df_handle_missing_val = handle_missing_val(
    df_not_handle_missing_val,
    features=missing_check_features,
    enabled=handle_missing,
    min_horses=min_non_missing_horses
)

dfの行数は 151128 
✅ 欠損処理を開始します...
✅ 欠損処理後のレコード数: 80496（元: 151128）


In [9]:
# 1. 処理対象データの準備
target_course_id = 1304
df_all = pd.read_csv('input_all_編集後.csv', encoding='utf-8')
df_course = df_all[df_all['course_id'] == target_course_id].copy()
# 2. レースIDと頭数（num_horses）のユニークな組み合わせを抽出
# 1レースにつき1レコードにするため、race_idとnum_horsesの重複を削除します。
# ※ df_allには既に num_horses が含まれていることが前提
df_races = df_course[['race_id', 'num_horses']].drop_duplicates()
# 3. 頭数（num_horses）ごとにレースIDの数をカウント
# つまり、その頭数で開催されたレースの数をカウント
head_count_stats = (
    df_races
    .groupby('num_horses')['race_id']
    .count()
    .rename('レース開催回数')
    .reset_index()
)
# 4. レース開催回数が多い順にソート
df_output = head_count_stats.sort_values(by='レース開催回数', ascending=False)
# 5. 結果の出力
print(f"### 🏟️ コースID: {target_course_id} における開催頭数別 レース回数 ###")
print(df_output.to_markdown(index=False, numalign="center", floatfmt=".0f"))

### 🏟️ コースID: 1304 における開催頭数別 レース回数 ###
|  num_horses  |  レース開催回数  |
|:------------:|:----------------:|
|      12      |       1220       |
|      11      |       510        |
|      10      |       403        |
|      9       |       113        |
|      8       |        30        |


In [29]:
# 1. データのインポート
import pandas as pd
import numpy as np

# === 設定パラメータ ===
target_course_id = 1304 # 例: コースID 15 を指定
num_horses = 12       # 例: 12頭立てのレースを指定

# race_dateをdatetime型に変換
df_all['race_date'] = pd.to_datetime(df_all['race_date'])

# 3. 全てのレコードに対して最終コーナー通過順位と正規化順位を計算 (過去データとして使用)
def get_final_corner_position(row):
    """ position_4, position_3, position_2, position_1 の順に確認し、最終コーナー通過順位を返す """
    if pd.notna(row['position_4']):
        return row['position_4']
    elif pd.notna(row['position_3']):
        return row['position_3']
    elif pd.notna(row['position_2']):
        return row['position_2']
    elif pd.notna(row['position_1']):
        return row['position_1']
    return np.nan

# df_allに対して計算
df_all['final_corner_position'] = df_all.apply(get_final_corner_position, axis=1)

# 正規化された順位 = 最終コーナー順位 / そのレースの頭数
df_all['normalized_position'] = (
    df_all['final_corner_position'] / df_all['num_horses']
)

# 4. 対象レースのフィルタリング（特徴量作成対象）
# ここで指定されたコースIDと頭数に該当するレコードに限定
df_target = df_all[
    (df_all['course_id'] == target_course_id) & 
    (df_all['num_horses'] == num_horses)
].copy()

if df_target.empty:
    print(f"⚠ 指定されたコースID: {target_course_id} かつ 頭数: {num_horses} のデータが存在しませんでした。")
    # 処理続行のために空のDFを定義
    df_feature_with_position = pd.DataFrame() 
    # 必要に応じてここでexit()

# 5. 過去の正規化された最終コーナー通過順位の平均値の計算（リーク防止）
# 過去のデータは df_all (全データ) を参照します
def calculate_past_position_avg_all(horse_id, race_date):
    # 対象馬の、今回のレース日より過去の df_all (全データ) をフィルタリング
    past_races = df_all[
        (df_all['horse_id'] == horse_id) & 
        (df_all['race_date'] < race_date)
    ]
    
    if past_races.empty:
        return np.nan
    else:
        # 過去の正規化された順位の平均を返す
        return past_races['normalized_position'].mean()

print("⏳ 過去の全レースデータに基づく最終コーナー位置取り平均の計算を開始します...")

# df_target のレコードに対して計算を適用
df_target['avg_past_normalized_position'] = df_target.apply(
    lambda row: calculate_past_position_avg_all(row['horse_id'], row['race_date']), 
    axis=1
)
print("✅ 計算完了。")


# 6. 位置取り傾向の特徴量化 (0.5を閾値として分類)
df_target['position_tendency'] = np.select(
    [
        df_target['avg_past_normalized_position'] < 0.5,  # 0.5未満: 前方傾向
        df_target['avg_past_normalized_position'] >= 0.5  # 0.5以上: 後方傾向
    ],
    [
        '前方傾向', 
        '後方傾向'
    ],
    default='データなし'
)

# 7. 結果の一部表示
print("\n### 📈 最終コーナー位置取り傾向の特徴量 (一部) ###")
print(f"対象レースの件数: {len(df_target)}")
print(df_target[['race_id', 'horse_id', 'race_date', 
                 'avg_past_normalized_position', 'position_tendency']].head(10).to_markdown())

# このdf_targetを、後続のモデル学習用データとして使用できます
df_feature_with_position = df_target.copy()

⏳ 過去の全レースデータに基づく最終コーナー位置取り平均の計算を開始します...
✅ 計算完了。

### 📈 最終コーナー位置取り傾向の特徴量 (一部) ###
対象レースの件数: 14640
|        | race_id   |   horse_id | race_date           |   avg_past_normalized_position | position_tendency   |
|-------:|:----------|-----------:|:--------------------|-------------------------------:|:--------------------|
| 219591 | 220408N03 | 2018110047 | 2022-04-08 00:00:00 |                       0.690965 | 後方傾向            |
| 219592 | 220408N03 | 2018102594 | 2022-04-08 00:00:00 |                       0.537955 | 後方傾向            |
| 219593 | 220408N03 | 2018104715 | 2022-04-08 00:00:00 |                       0.366667 | 前方傾向            |
| 219594 | 220408N03 | 2018100235 | 2022-04-08 00:00:00 |                       0.568114 | 後方傾向            |
| 219595 | 220408N03 | 2018101408 | 2022-04-08 00:00:00 |                       0.70115  | 後方傾向            |
| 219596 | 220408N03 | 2018106067 | 2022-04-08 00:00:00 |                       0.664162 | 後方傾向            |
| 219597 | 220408N03 |

In [30]:
# 1. 処理対象データの準備
# 前のステップで作成された df_feature_with_position を使用
df_analysis = df_feature_with_position.copy()

# 2. レースごとの前方傾向/後方傾向の頭数カウント
race_counts = (
    df_analysis
    .groupby('race_id')['position_tendency']
    .value_counts()
    .unstack(fill_value=0) # 前方傾向、後方傾向の頭数を列に展開
    .rename(columns={'前方傾向': 'count_forward', '後方傾向': 'count_backward', 'データなし': 'count_nan'})
)

# 3. レースごとの全体の頭数を計算
race_counts['total_runners'] = race_counts['count_forward'] + race_counts['count_backward'] + race_counts['count_nan']

# 4. レースにおける前方傾向の割合（パーセンテージ）を計算し、グループ化のためのビンを作成
race_counts['forward_ratio'] = (race_counts['count_forward'] / race_counts['total_runners'])

# 割合に基づいたグループを作成 (例: 0-20%, 20-40%, 40-60%, 60-80%, 80-100%)
bins = np.arange(0, 1.01, 0.2)
labels = [f'{int(l*100)}-{int(r*100)}%' for l, r in zip(bins[:-1], bins[1:])]
race_counts['ratio_group'] = pd.cut(
    race_counts['forward_ratio'], 
    bins=bins, 
    labels=labels, 
    right=True, 
    include_lowest=True
)

# 5. 元のデータフレームにレースごとの「前方傾向の割合グループ」をマージ
df_analysis = df_analysis.merge(
    race_counts[['ratio_group']],
    left_on='race_id',
    right_index=True,
    how='left'
)

# 6. '勝ち'のフラグを作成 (finish_rankが1なら1, それ以外は0)
df_analysis['is_win'] = (df_analysis['finish_rank'] == 1).astype(int)

# 7. グループ別・傾向別の集計
# ratio_groupとposition_tendencyでグループ化し、勝率（is_winの平均）を計算
grouped_win_rate = (
    df_analysis
    .groupby(['ratio_group', 'position_tendency'])
    .agg(
        win_rate=('is_win', 'mean'),
        total_starters=('is_win', 'count')
    )
    .reset_index()
)

# 8. 不要な 'データなし' 傾向を除外
grouped_win_rate = grouped_win_rate[grouped_win_rate['position_tendency'].isin(['前方傾向', '後方傾向'])].copy()

# 9. 結果の整形と出力
grouped_win_rate['勝率(%)'] = (grouped_win_rate['win_rate'] * 100).round(2)

df_output = (
    grouped_win_rate[['ratio_group', 'position_tendency', '勝率(%)', 'total_starters']]
    .pivot(index='ratio_group', columns='position_tendency', values=['勝率(%)', 'total_starters'])
)

# 列名の整理
df_output.columns = [f'{tendency}の{metric}' for metric, tendency in df_output.columns]

print("### 📊 レース内の前方傾向割合別 傾向別勝率 ###")
print("※ '前方傾向の割合' は、そのレースにおける前方傾向の馬の頭数の割合を示します。")
print(df_output.to_markdown(floatfmt=".2f"))

### 📊 レース内の前方傾向割合別 傾向別勝率 ###
※ '前方傾向の割合' は、そのレースにおける前方傾向の馬の頭数の割合を示します。
| ratio_group   |   前方傾向の勝率(%) |   後方傾向の勝率(%) |   前方傾向のtotal_starters |   後方傾向のtotal_starters |
|:--------------|--------------------:|--------------------:|---------------------------:|---------------------------:|
| 0-20%         |               16.80 |                5.79 |                     244.00 |                    1658.00 |
| 20-40%        |               16.84 |                3.42 |                    1051.00 |                    2313.00 |
| 40-60%        |               12.61 |                3.48 |                    3212.00 |                    3162.00 |
| 60-80%        |               10.60 |                2.51 |                    1264.00 |                     518.00 |
| 80-100%       |                9.11 |                3.39 |                     406.00 |                      59.00 |


C:\Users\ryo\AppData\Local\Temp\ipykernel_12268\903565676.py:46: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['ratio_group', 'position_tendency'])


In [15]:
# 1. 処理対象データの準備
# 前のステップで作成された df_feature_with_position を使用
df_analysis = df_feature_with_position.copy()

# 2. '勝ち'のフラグを作成
df_analysis['is_win'] = (df_analysis['finish_rank'] == 1).astype(int)

# 3. 'position_tendency'に全体を表すカテゴリを追加
# 全ての馬に 'ALL' というダミーカテゴリを割り当てる
df_analysis['position_tendency_all'] = df_analysis['position_tendency']
df_analysis['position_tendency_all_with_all'] = df_analysis['position_tendency_all'].where(
    df_analysis['position_tendency_all'].isin(['前方傾向', '後方傾向']), 'データなし'
)

# 4. 前方傾向/後方傾向/データなし、および人気順位でグループ化し、勝率とサンプル数を計算
grouped_stats_tendency = (
    df_analysis
    .groupby(['position_tendency_all_with_all', 'popularity'])
    .agg(
        win_rate=('is_win', 'mean'),
        total_starters=('is_win', 'count')
    )
    .rename(index={'position_tendency_all_with_all': 'position_tendency'}) # 列名を元に戻す
    .reset_index()
)

# 5. 全体（ALL）の勝率を計算
grouped_stats_all = (
    df_analysis
    .groupby('popularity')
    .agg(
        win_rate=('is_win', 'mean'),
        total_starters=('is_win', 'count')
    )
    .reset_index()
)
# 全体に 'ALL' のラベルを付与
grouped_stats_all['position_tendency_all_with_all'] = 'ALL'

# 6. 傾向別の統計と全体の統計を結合
df_combined_stats = pd.concat([grouped_stats_tendency, grouped_stats_all], ignore_index=True)

# 7. 不要な 'データなし' 傾向を除外（ALLは残す）
df_combined_stats = df_combined_stats[df_combined_stats['position_tendency_all_with_all'].isin(['前方傾向', '後方傾向', 'ALL'])].copy()

# 8. 勝率をパーセンテージに変換
df_combined_stats['勝率(%)'] = (df_combined_stats['win_rate'] * 100).round(2)

# 9. 結果を整形するためピボット
df_output = (
    df_combined_stats[['position_tendency_all_with_all', 'popularity', '勝率(%)', 'total_starters']]
    .pivot(index='popularity', columns='position_tendency_all_with_all', values=['勝率(%)', 'total_starters'])
)

# 10. 列名の整理と順序の調整
# MultiIndexをFlatにする
df_output.columns = [f'{tendency}_{metric}' for metric, tendency in df_output.columns]

# 列順の調整: 勝率 → サンプル数（前方, 後方, ALL の順）
cols_order_win_rate = [f'{t}_勝率(%)' for t in ['前方傾向', '後方傾向', 'ALL']]
cols_order_starters = [f'{t}_total_starters' for t in ['前方傾向', '後方傾向', 'ALL']]
final_cols_order = []
for wr, ts in zip(cols_order_win_rate, cols_order_starters):
    final_cols_order.extend([wr, ts])
    
df_output = df_output.reindex(columns=final_cols_order)

# 11. 結果の出力
print(f"### 🏆 コースID: {target_course_id}, 頭数: {num_horses} における傾向別・ALL別 人気別 勝率 ###")
print("※ 'ALL' は、位置取り傾向を区別しない場合の全体勝率とサンプル数を示します。")
print(df_output.to_markdown(floatfmt=".2f"))

### 🏆 コースID: 1304, 頭数: 12 における傾向別・ALL別 人気別 勝率 ###
※ 'ALL' は、位置取り傾向を区別しない場合の全体勝率とサンプル数を示します。
|   popularity |   前方傾向_勝率(%) |   前方傾向_total_starters |   後方傾向_勝率(%) |   後方傾向_total_starters |   ALL_勝率(%) |   ALL_total_starters |
|-------------:|-------------------:|--------------------------:|-------------------:|--------------------------:|--------------:|---------------------:|
|         1.00 |              42.94 |                    892.00 |              30.17 |                    179.00 |         41.64 |              1220.00 |
|         2.00 |              20.83 |                    816.00 |              21.36 |                    295.00 |         20.90 |              1220.00 |
|         3.00 |              13.13 |                    739.00 |               9.74 |                    390.00 |         12.30 |              1220.00 |
|         4.00 |               7.51 |                    692.00 |               7.96 |                    452.00 |          7.94 |              1222.00 |
|     

In [17]:
# 1. 処理対象データの準備
# 前のステップで作成された df_feature_with_position を使用
df_analysis = df_feature_with_position.copy()

# === 設定パラメータの追加 ===
# 例: 人気1〜3番の馬をまとめて分析
target_popularities = [1] 

# 2. '勝ち'のフラグを作成 (finish_rankが1なら1, それ以外は0)
df_analysis['is_win'] = (df_analysis['finish_rank'] == 1).astype(int)

# 3. 指定された人気順位の馬のみをフィルタリング
df_filtered = df_analysis[df_analysis['popularity'].isin(target_popularities)].copy()

if df_filtered.empty:
    print(f"⚠ 指定された人気順位 {target_popularities} の馬はデータセットに存在しませんでした。")
    # 処理終了
    # exit() 
    raise ValueError("指定された人気順位のデータが見つかりません。")

# 4. 位置取り傾向（position_tendency）と馬番（horse_number）でグループ化し、勝率とサンプル数を計算
grouped_stats = (
    df_filtered
    .groupby(['position_tendency', 'horse_number'])
    .agg(
        win_rate=('is_win', 'mean'),
        total_starters=('is_win', 'count')
    )
    .reset_index()
)

# 5. 不要な 'データなし' 傾向を除外
grouped_stats = grouped_stats[grouped_stats['position_tendency'].isin(['前方傾向', '後方傾向'])].copy()

# 6. 勝率をパーセンテージに変換
grouped_stats['勝率(%)'] = (grouped_stats['win_rate'] * 100).round(2)

# 7. 結果を整形するためピボット
df_output = (
    grouped_stats[['position_tendency', 'horse_number', '勝率(%)', 'total_starters']]
    .pivot(index='horse_number', columns='position_tendency', values=['勝率(%)', 'total_starters'])
)

# 8. 列名の整理と順序の調整
df_output.columns = [f'{tendency}_{metric}' for metric, tendency in df_output.columns]

cols_order = []
for tendency in ['前方傾向', '後方傾向']:
    cols_order.append(f'{tendency}_勝率(%)')
    cols_order.append(f'{tendency}_total_starters')

df_output = df_output.reindex(columns=cols_order)

# 9. 結果の出力
popularity_str = ', '.join(map(str, target_popularities))
print(f"### 🏆 コースID: {target_course_id}, 頭数: {num_horses}, 人気: [{popularity_str}] の傾向別・馬番別 勝率 ###")
print("※ 勝率は、指定された人気（合算）における1着になる確率を示します。")
print(df_output.to_markdown(floatfmt=".2f"))

### 🏆 コースID: 1304, 頭数: 12, 人気: [1] の傾向別・馬番別 勝率 ###
※ 勝率は、指定された人気（合算）における1着になる確率を示します。
|   horse_number |   前方傾向_勝率(%) |   前方傾向_total_starters |   後方傾向_勝率(%) |   後方傾向_total_starters |
|---------------:|-------------------:|--------------------------:|-------------------:|--------------------------:|
|              1 |              34.62 |                     52.00 |              57.14 |                      7.00 |
|              2 |              27.69 |                     65.00 |              25.00 |                     12.00 |
|              3 |              33.33 |                     72.00 |              17.65 |                     17.00 |
|              4 |              34.18 |                     79.00 |              31.58 |                     19.00 |
|              5 |              48.39 |                     62.00 |              46.15 |                     13.00 |
|              6 |              41.33 |                     75.00 |              54.55 |                     11.00 

In [19]:
## 天気（weather_id）別・人気順位別 1着勝率の計算

# 1. 処理対象データの準備
df_win_rate = df.copy()

# 2. '勝ち'のフラグを作成 (finish_rankが1なら1, それ以外は0)
df_win_rate['is_win'] = (df_win_rate['finish_rank'] == 1).astype(int)

# 3. 天気（weather_id）と人気順位（popularity）ごとに勝率（is_winの平均）を計算
grouped_stats = (
    df_win_rate
    .groupby(['weather_name', 'popularity'])
    .agg(
        win_rate=('is_win', 'mean'),
        # 各グループのサンプル数を計算（勝率の信頼性の目安）
        total_starters_group=('is_win', 'count') 
    )
    .reset_index()
)

# 4. 勝率を小数点以下2桁のパーセンテージに変換
grouped_stats['win_rate_percent'] = (grouped_stats['win_rate'] * 100).round(2)

# 5. 勝率データ（パーセンテージ）をピボットし、人気順位を列に展開
df_win_rate_pivot = (
    grouped_stats
    .pivot(index='weather_name', columns='popularity', values='win_rate_percent')
    .fillna(0) # データがない組み合わせは0%で埋める
    .rename_axis(None, axis=1) # 列名の軸ラベルを削除
)

# 6. 各weather_idごとの合計出走頭数（サンプルサイズ）を計算
total_starters_by_weather = (
    df_win_rate
    .groupby('weather_name')['is_win']
    .count()
    .rename('合計出走頭数')
)

# 7. ピボット結果と合計出走頭数を結合
df_output = df_win_rate_pivot.join(total_starters_by_weather).fillna(0)

# 8. 合計出走頭数が多い順にソート
df_output = df_output.sort_values(by='合計出走頭数', ascending=False)

# 9. 結果の出力
print("### 🌦️ 天気（weather_id）別・人気順位別 1着勝率（%） ###")
print("※ 表中の数値は、その人気順位の馬が1着になる確率（%）を示します。")
print(df_output.to_markdown(numalign="center", floatfmt=".2f"))

### 🌦️ 天気（weather_id）別・人気順位別 1着勝率（%） ###
※ 表中の数値は、その人気順位の馬が1着になる確率（%）を示します。
| weather_name   |   1   |   2   |   3   |  4   |  5   |  6   |  7   |  8   |  9   |  10  |  11  |  12  |  合計出走頭数  |
|:---------------|:-----:|:-----:|:-----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:--------------:|
| 晴             | 40.80 | 19.08 | 11.95 | 9.07 | 5.96 | 4.37 | 3.00 | 2.26 | 1.39 | 1.12 | 0.66 | 0.36 |    87024.00    |
| 曇             | 38.45 | 22.45 | 12.44 | 8.11 | 5.97 | 4.62 | 3.17 | 2.12 | 1.07 | 0.97 | 0.42 | 0.33 |    48036.00    |
| 雨             | 38.43 | 19.72 | 13.00 | 8.75 | 6.27 | 5.61 | 3.51 | 1.75 | 1.29 | 1.01 | 0.83 | 0.09 |    13020.00    |
| 小雨           | 41.71 | 19.43 | 11.85 | 9.95 | 5.21 | 5.19 | 3.33 | 0.95 | 0.95 | 0.95 | 0.47 | 0.00 |    2532.00     |
| 雪             | 64.00 | 4.00  | 4.00  | 8.00 | 8.00 | 4.00 | 4.00 | 0.00 | 4.00 | 0.00 | 0.00 | 0.00 |     300.00     |
| 小雪           | 50.00 | 22.22 | 16.67 | 5.56 | 5.56 | 0.00 | 0.00 | 0.00 |

In [21]:
## コース別・年別 レース開催数の集計（合計ソート版）

# 1. 処理対象データの準備
df_yearly_stats = df.copy()

# 2. 'race_date'から開催年を抽出
# race_dateは既にdatetime型に変換されている前提
df_yearly_stats['開催年'] = df_yearly_stats['race_date'].dt.year

# 3. コースIDと開催年の組み合わせでレースIDのユニーク数をカウント
yearly_race_counts = (
    df_yearly_stats
    .groupby(['course_id', '開催年'])['race_id']
    .nunique()
    .rename('レース開催回数')
    .reset_index()
)

# 4. '開催年'を列に展開（ピボット）
df_pivot = (
    yearly_race_counts
    .pivot(index='course_id', columns='開催年', values='レース開催回数')
    .fillna(0) # データがない年・コースの組み合わせは0で埋める
)

# 5. 合計開催数（全ての年の合計）の列を追加
df_pivot['合計開催回数'] = df_pivot.sum(axis=1)

# 6. 合計開催回数が多い順にソート
df_output = df_pivot.sort_values(by='合計開催回数', ascending=False)

# 7. 結果の出力
print("### 🏟️ コースID別・年別 レース開催回数（合計ソート） ###")
print("※ 合計開催回数が多い順にソートされています。")
print(df_output.to_markdown(stralign="center", numalign="center", floatfmt=".0f"))

### 🏟️ コースID別・年別 レース開催回数（合計ソート） ###
※ 合計開催回数が多い順にソートされています。
|  course_id  |  2020  |  2021  |  2022  |  2023  |  2024  |  合計開催回数  |
|:-----------:|:------:|:------:|:------:|:------:|:------:|:--------------:|
|    1304     |   0    |   0    |  324   |  435   |  461   |      1220      |
|     202     |  184   |  196   |  186   |  182   |  253   |      1001      |
|     102     |  212   |  205   |  142   |  130   |  132   |      821       |
|     502     |  135   |  121   |  122   |  132   |  136   |      646       |
|     901     |  131   |  118   |  149   |  109   |  113   |      620       |
|    1001     |  113   |  126   |  113   |  107   |  121   |      580       |
|     602     |   71   |   92   |   71   |  135   |  128   |      497       |
|     501     |  134   |  100   |   83   |   76   |   71   |      464       |
|     103     |  101   |   90   |   74   |   74   |   63   |      402       |
|    1400     |   52   |   81   |   43   |  110   |  113   |      399       |
|    1402 

In [23]:
## 騎乗回数による勝率の比較

# 1. データフレームの準備と「勝ち」のフラグ作成
df_temp = df_not_handle_missing_val.copy()
# finish_rankが1なら1（勝ち）、それ以外は0
df_temp['is_win'] = (df_temp['finish_rank'] == 1).astype(int)

# 2. 騎手IDと馬IDの組み合わせ（コンビ）を識別
df_temp['jockey_horse_combo'] = df_temp['jockey_id'].astype(str) + '_' + df_temp['horse_id'].astype(str)

# 3. コンビごとのレース日を昇順に並べ、各レースが何回目の騎乗かをカウント
df_temp['jockey_horse_race_count'] = (
    df_temp
    .sort_values(by=['jockey_horse_combo', 'race_date'])
    .groupby('jockey_horse_combo')
    .cumcount() + 1 # 1回目の騎乗から始める
)

# 4. 騎乗回数に応じてグループ分け
# 1回目の騎乗
is_first_ride = (df_temp['jockey_horse_race_count'] == 1)
# 2回目以降の騎乗
is_subsequent_ride = (df_temp['jockey_horse_race_count'] >= 2)

# 5. グループごとの勝率（is_winの平均値）を計算
# 勝率 = is_winの合計 / サンプル数 = is_winの平均値
win_rate_first_ride = df_temp[is_first_ride]['is_win'].mean()
win_rate_subsequent_ride = df_temp[is_subsequent_ride]['is_win'].mean()

# 6. 結果の表示
print("### 🥇 騎乗回数別 勝率の比較 🥇 ###")
print(f"対象レコード数: {len(df_temp)} 件")
print("-" * 40)
print(f"① 初めて騎乗したレースの勝率 (N={is_first_ride.sum()}): {win_rate_first_ride:.4f} ({win_rate_first_ride*100:.2f}%)")
print(f"② 2回目以降に騎乗したレースの勝率 (N={is_subsequent_ride.sum()}): {win_rate_subsequent_ride:.4f} ({win_rate_subsequent_ride*100:.2f}%)")
print("-" * 40)


### 🥇 騎乗回数別 勝率の比較 🥇 ###
対象レコード数: 151128 件
----------------------------------------
① 初めて騎乗したレースの勝率 (N=78891): 0.0854 (8.54%)
② 2回目以降に騎乗したレースの勝率 (N=72237): 0.0812 (8.12%)
----------------------------------------


In [27]:
## コース別統計の算出とCSV出力

# 1. 処理対象データの準備
df_stats = df_not_handle_missing_val.copy()

# 2. 1着馬のみを抽出
df_winners = df_stats[df_stats['finish_rank'] == 1].copy()

# 3. コースごとの統計情報を集計
# レース開催回数: 'race_id'のユニーク数をカウント
race_counts = (
    df_stats
    .groupby('course_id')['race_id']
    .nunique()
    .rename('レース開催回数')
)

# 4. 1着馬の人気（人気順位）ごとのカウント
popularity_counts = (
    df_winners
    .groupby(['course_id', 'popularity'])['race_id']
    .count()
    .rename('1着人気カウント')
    .unstack(fill_value=0) # 人気順位を列に展開
)

# 5. 結果の結合
df_result = race_counts.to_frame()
df_result = df_result.join(popularity_counts, how='left').fillna(0)

# 6. 1着馬の総数を計算
df_result['1着馬総数'] = df_result.iloc[:, 1:].sum(axis=1)

# 7. 各人気カウントの割合を計算
# 割合 = (各人気カウント) / (1着馬総数)
count_cols_int = [c for c in popularity_counts.columns]
for col in count_cols_int:
    # 列名を分かりやすくするために整数型から文字列に変換
    df_result[f'人気{int(col)}_カウント'] = df_result[col] 
    df_result[f'人気{int(col)}_割合(%)'] = (df_result[col] / df_result['1着馬総数']) * 100

# 8. 不要な列の削除と列名の整理
cols_to_drop = [c for c in df_result.columns if isinstance(c, int)] # 元の整数列を削除
df_final_stats = df_result.drop(columns=cols_to_drop)

# 9. インデックス（course_id）を列に戻し、レース開催回数でソート
df_final_stats = (
    df_final_stats
    .reset_index()
    .sort_values(by='レース開催回数', ascending=False)
)

# 10. CSVファイルとして出力
df_final_stats.to_csv('C:\\Users\\ryo\\Downloads\\course_stats_by_popularity.csv', index=False, encoding='cp932')
print("✅ CSVファイルの出力が完了しました。")

✅ CSVファイルの出力が完了しました。


filtered_dfの行数は 33144 
15432 15432 17712 17712


C:\Users\ryo\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:53:07] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


finish_rank を含めた全段階の予測結果CSVを出力しました。
▼ モデル1（上位6頭予測）の的中率
  予測数: 8856, 的中数: 5775, 的中率: 0.6521（65.21%）


📊 人気ごとの1着予想的中率＆回収率（pred_prob_m1最大の馬を1着予想として集計）
    popularity  total_pred  hits  accuracy       roi
0            1         650   290  0.446154  0.809231
1            2         306    59  0.192810  0.710131
2            3         185    31  0.167568  0.975135
3            4         119    17  0.142857  1.045378
4            5          75     7  0.093333  0.860000
5            6          55     3  0.054545  0.780000
6            7          32     1  0.031250  0.668750
7            8          26     1  0.038462  0.665385
8            9          11     0  0.000000  0.000000
9           10          10     0  0.000000  0.000000
10          11           5     0  0.000000  0.000000
11          12           2     0  0.000000  0.000000
